<h2> Segmentación de imagenes</h2>
<h3> Computer Vision con Python - OpenCV</h3> 

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

In [2]:
video_path = "../videos/deteccion_tienda.mp4"

In [3]:
cap = cv2.VideoCapture(video_path)

# suscribe to the YOLO model

bg_subtractor = cv2.createBackgroundSubtractorMOG2(
    history=500,
    varThreshold=16,
    detectShadows=True
)

heatmap_refined = None

# Cargue del modelo YOLO para segmentation
model = YOLO("yolo11n-seg")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Iniciar el procesamiento de la imagen
    if heatmap_refined is None:
        heatmap_refined = np.zeros(frame.shape[:2], dtype=np.float32)
        
    # 1 --- Susrección del fondo
    fgmask = bg_subtractor.apply(frame)
    _, fgmask = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
    
    # 2 --- Detección de objetos con YOLO
    results = model(frame, verbose=False)[0]
    
    # Crear camara vacia para acumular las imagenes.
    segmentation_mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    
    if results.masks is not None:
        # Extraer las mascaras y las clases.
        masks = results.masks.data.cpu().numpy() if hasattr(results.masks.data, 'cpu') else results.masks.data
        classes = results.boxes.cls.cpu().numpy() if hasattr(results.boxes.cls, 'cpu') else results.boxes.cls
        
        for mask, cls in zip(masks, classes):
            if int(cls) == 0:
                mask_bin = (mask > 0.5).astype(np.uint8) * 255
                mask_bin_resized = cv2.resize(mask_bin, (segmentation_mask.shape[1], segmentation_mask.shape[0]), interpolation=cv2.INTER_NEAREST)
                segmentation_mask = cv2.bitwise_or(segmentation_mask, mask_bin_resized)
                
    # 3 Combinación de la máscara de segmentación con el fondo
    refined_mask = cv2.bitwise_and(segmentation_mask, fgmask)
    
    # Acumular la mascara refinada
    heatmap_refined = cv2.add(heatmap_refined, refined_mask.astype(np.float32))
    
    # Visualización del heatmap
    cv2.imshow("Frame Original", frame)
    cv2.imshow("Mascara de movimiento (FG)", fgmask)
    cv2.imshow("Segmentación personas", segmentation_mask)
    cv2.imshow("Mascara refinada", refined_mask)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

100%|██████████| 5.90M/5.90M [00:00<00:00, 7.03MB/s]
